# Introduction to Databases in Python

In this notebook, you'll learn the basics of using SQL with Python. This will be useful because databases are ubiquitous and data scientists, analysts, and engineers must interact with them constantly. The Python SQL toolkit **SQLAlchemy** provides an accessible and intuitive way to query, build, and write to essential databases, including `SQLite`, `MySQL`, and `PostgreSQL`.